In [109]:
import pandas as pd
import numpy as np
import sklearn
import json
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import svm
from tqdm import tqdm

In [103]:
f = open('/users/monaal/downloads/all.txt')
predictions = json.load(f)
f = open('/users/monaal/downloads/latent_vector_all.txt')
latent_vectors = json.load(f)

In [104]:
latent_vectors_df = pd.DataFrame.from_dict(latent_vectors,orient='index')

In [178]:
latent_vectors_df=latent_vectors_df.reset_index(drop=True)
latent_vectors_df.columns=['latent_space']


In [106]:
df = pd.DataFrame(predictions,columns=['feature_1','feature_2','feature_3','feature_4','feature_5','feature_6','feature_7','feature_8','feature_9','feature_10'])

In [179]:
df_temp=pd.concat([df,latent_vectors_df],axis=1)


In [157]:
dic={}
for i in tqdm(range(1,11)):
        fet = 'feature_'+str(i)
        df_1=df_temp.sort_values(by=fet,ascending=False)
        df_head=df_1.head(3000)
        df_head[fet]=[1]*3000
        df_tail=df_1.tail(3000)
        df_tail[fet]=[0]*3000
        df_final=pd.concat([df_head,df_tail])
        df_final=df_final.reset_index(drop=True)
        y=df_final[fet]
        X=df_final['latent_space']
        X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
        clf = svm.SVC(kernel='linear')
        clf.fit(np.concatenate(X_train.values).reshape(X_train.shape[0],-1),y_train.values)
        y_pred=clf.predict(np.concatenate(X_test.values).reshape(X_test.shape[0],-1))
        dic[i] = clf.coef_/np.linalg.norm(clf.coef_)
        

  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-157-06f1418edef3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_head[fet]=[1]*3000
<ipython-input-157-06f1418edef3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tail[fet]=[0]*3000
 10%|█         | 1/10 [01:15<11:17, 75.29s/it]<ipython-input-157-06f1418edef3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [175]:
def linear_interpolate(latent_code,
                       boundary,
                       start_distance=-3.0,
                       end_distance=3.0,
                       steps=10):

  linspace = np.linspace(start_distance, end_distance, steps)
  if len(np.asarray(latent_code).shape) == 2:
    linspace = linspace - np.asarray(latent_code).dot(boundary.T)
    linspace = linspace.reshape(-1, 1).astype(np.float32)
    return np.asarray(latent_code) + linspace * np.asarray(boundary)
  if len(np.asarray(latent_code).shape) == 3:
    linspace = linspace.reshape(-1, 1, 1).astype(np.float32)
    return np.asarray(latent_code) + linspace * np.asarray(boundary).reshape(1, 1, -1)
  raise ValueError(f'Input `latent_code` should be with shape '
                   f'[1, latent_space_dim] or [1, N, latent_space_dim] for '
                   f'W+ space in Style GAN!\n'
                   f'But {np.asarray(latent_code).shape} is received.')

In [261]:
def project_boundary(primal, args):
 

     
        for cond_boundary in args:
            cond_boundary.shape

            cond_boundaries = np.squeeze(np.asarray(args))
            A = np.matmul(cond_boundaries, cond_boundaries.T)
            B = np.matmul(cond_boundaries, primal.T)
            x = np.linalg.solve(A, B)
            new = primal - (np.matmul(x.T, cond_boundaries))
        return new / np.linalg.norm(new)

In [262]:
a=[]
for i in range(2,11):
    a.append(dic[i])

In [263]:
new_bound = project_boundary((dic[1]),a)

In [267]:
vals = []
for i in range(1,11):
    new_dic = np.asarray([v for k,v in dic.items() if i!=k])
    vals.append(project_boundary((dic[i]),new_dic))
    

    

In [ ]:
for i in range(1,11):
    linear_interpolate(np.asarray(df_temp['latent_space'][0]).reshape(1,512),np.asarray(dic[1]),start_distance=-3.0,end_distance=3.0,
                           steps=10).shape

In [270]:
len(df_temp['latent_space'])

30000

In [285]:
temp = []
for j in range(10):
    df_temp["newly_created_latent_space_"+str(j)] = df_temp['latent_space'].map(lambda x: (linear_interpolate(np.asarray(x).reshape(1,512),np.asarray(vals[j]))))
        
    

In [274]:
np.array(temp).shape

(1, 10, 512)

In [290]:
df_temp[:1000].to_pickle("./latent_spaces_created.pkl")

In [288]:
df_temp[:100]

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,newly_created_latent_space_0,newly_created_latent_space_1,newly_created_latent_space_2,newly_created_latent_space_3,newly_created_latent_space_4,newly_created_latent_space_5,newly_created_latent_space_6,newly_created_latent_space_7,newly_created_latent_space_8,newly_created_latent_space_9
0,0.272510,0.016838,0.019215,0.959503,0.029840,0.872176,0.013826,0.010904,0.004795,0.873599,...,"[[0.27452762361800365, 1.857480903624631, -0.1...","[[0.4735279417905355, 2.1739595311453024, 0.00...","[[0.47216007489150524, 2.137640209331166, 0.04...","[[0.43723067501618096, 2.2215615137920444, -0....","[[0.6052786138824744, 2.1301836869597124, 0.17...","[[0.4546020761170424, 2.0076634760738616, -0.1...","[[0.2929941177227974, 2.2420785055203742, -0.2...","[[0.5671346995894757, 2.185513642683394, -0.04...","[[0.17686405987850895, 1.8967652627121172, -0....","[[0.23356992609184624, 1.7078577653575853, -0...."
1,0.016161,0.080971,0.387027,0.543562,0.954619,0.041516,0.788839,0.523966,0.013405,0.262806,...,"[[0.42496281367658506, 1.2400361386477174, 0.5...","[[0.5465757301646137, 1.4378669041411494, 0.60...","[[0.5865862941074947, 1.416537252882472, 0.695...","[[0.585014855283796, 1.5851429436983764, 0.426...","[[0.7495084704239474, 1.407677103151207, 0.851...","[[0.5753037291551434, 1.2533595591780318, 0.48...","[[0.38091031658503044, 1.5390190207600674, 0.3...","[[0.6748615249549453, 1.465026831776575, 0.588...","[[0.3166530910286084, 1.2133161730048747, 0.55...","[[0.39802599400093924, 1.1427331404911012, 0.4..."
2,0.426919,0.183998,0.116419,0.943441,0.373614,0.041903,0.678499,0.069998,0.037316,0.024321,...,"[[0.8974063556213359, -0.7145897588368941, 0.3...","[[1.1042664167141518, -0.32428047676554855, 0....","[[1.1222239787480734, -0.34671242832531285, 0....","[[1.1499721339086213, -0.15829112038599508, 0....","[[1.2916521350061196, -0.3552028699467597, 0.7...","[[1.1417743137277327, -0.5353241943741491, 0.3...","[[0.9324875087634593, -0.22084072080908343, 0....","[[1.0883049400832236, -0.3375040651084278, 0.4...","[[0.8361704061118587, -0.5892262836001638, 0.4...","[[1.017095763346917, -0.4507479828886236, 0.41..."
3,0.088514,0.240854,0.397162,0.648786,0.183886,0.096604,0.377391,0.152581,0.424727,0.028166,...,"[[-1.476774072749861, -0.7970960702047967, 0.6...","[[-1.2467464558840817, -0.464553132913248, 0.7...","[[-1.2754553780374138, -0.5147483375703903, 0....","[[-1.2654341562400664, -0.33493744083012456, 0...","[[-1.0245749914103552, -0.5267427041944839, 1....","[[-1.2720533994517624, -0.6971987014021007, 0....","[[-1.4955380497606836, -0.3716472677306275, 0....","[[-1.2094310372802162, -0.47410917177944056, 0...","[[-1.6104262685852075, -0.7963423083419975, 0....","[[-1.4928648585862763, -0.8810600367094991, 0...."
4,0.640686,0.705744,0.343783,0.880950,0.059896,0.118746,0.055666,0.038436,0.014525,0.085006,...,"[[0.3540723289906458, -0.3682578953701393, 1.2...","[[0.5931389979384994, 0.001244864180854753, 1....","[[0.6364845155002776, -0.04535254267827675, 1....","[[0.5408567782920929, 0.04992209466917208, 1.2...","[[0.6681128393850824, -0.051071399737189715, 1...","[[0.5712274267551554, -0.2157924401299792, 1.2...","[[0.29475403255505855, 0.1570768168644782, 1.0...","[[0.680894419959168, 0.007231877260468325, 1.3...","[[0.2984481431085368, -0.2600165169947662, 1.2...","[[0.33214528300580926, -0.473681783599616, 1.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.478787,0.004566,0.198202,0.093101,0.120611,0.008411,0.988831,0.957118,0.160614,0.017791,...,"[[0.8080328010582487, 0.7720625462748902, -0.7...","[[0.8988623556536839, 0.8677823266309359, -0.6...","[[0.9227248631032594, 0.8387889834923135, -0.6...","[[0.8583397740409989, 0.8820637619532351, -0.7...","[[1.0996257756541912, 0.8293853680432384, -0.4...","[[0.8832948890090286, 0.7369157252354352, -0.7...","[[0.7294200527418025, 0.94